In [28]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,execute_insert_queries,extract_data_into_df
from df_handler import load_data_into_dataframe
from lookups import FileType

db_session = create_connection()

In [29]:

# stg_users_dataframe = pd.DataFrame({
#     "age": [25, 30, 35],
#     "gender": ['Male', 'Female', 'Male'],
#     "fav_music_genre": ['Pop', 'Rock', 'Hip-Hop'],
#     "music_time_slot": ['Morning', 'Evening', 'Afternoon'],
#     "music_lis_frequency": ['Daily', 'Weekly', 'Monthly'],
#     "music_satisfaction": ['High', 'Medium', 'Low']
# })

# stg_songs_dataframe = pd.DataFrame({
#     "track_id": [1, 2, 3],
#     "track_name": ["Song1", "Song2", "Song3"],
#     "track_artist": ["Artist1", "Artist2", "Artist3"],
#     "track_popularity": [80, 90, 70],
#     "track_album_id": [101, 102, 103],
#     "track_album_name": ["Album1", "Album2", "Album3"],
#     "track_album_release_date": ["2022-01-01", "2022-02-01", "2022-03-01"],
#     "playlist_name": ["Playlist1", "Playlist2", "Playlist3"],
#     "playlist_id": [201, 202, 203],
#     "playlist_genre": ["Genre1", "Genre2", "Genre3"],
#     "energy": [0.8, 0.7, 0.6],
#     "key": [1, 2, 3],
#     "loudness": [-5.0, -6.0, -7.0],
#     "mode": [0, 1, 0],
#     "speechiness": [0.1, 0.2, 0.3],
#     "valence": [0.7, 0.6, 0.5],
#     "tempo": [120, 130, 140],
#     "duration_ms": [240000, 250000, 260000]
# })




In [30]:

# stg_users_table_name = "musicschema"
# stg_users_schema_name = 'public' 
# stg_users_create_statement = return_create_statement_from_dataframe(
#     stg_users_dataframe, stg_users_schema_name, stg_users_table_name)

# query = insert_statements_from_dataframe(stg_users_dataframe,stg_users_schema_name ,stg_users_table_name)

# for statement in query:
#     execute_query(db_session, statement)
#     print(statement)
    
# stg_songs_table_name = 'musicschema'
# stg_songs_schema_name = 'public'  
# stg_songs_create_statement = return_create_statement_from_dataframe(
#     stg_songs_dataframe, stg_songs_schema_name, stg_songs_table_name)

# execute_query(conn= db_session, query= stg_users_create_statement)

# execute_query(conn= db_session, query= stg_songs_create_statement)

In [38]:
datatype=FileType.CSV
datapath="C:/Users/Admin/Desktop/dataeng_fix/LatestComm/songs_dataset.csv"

dfdata=extract_data_into_df(datatype,datapath)
print(dfdata)

                     track_id  \
0      6f807x0ima9a1j3VPbc7VN   
1      0r7CVbZTWZgbTCYdfa2P31   
2      1z1Hg7Vb0AhHDiEmnDE79l   
3      75FpbthrwQmzHlBJLuGdC7   
4      1e8PAfcKUYoKkxPhrHqw4x   
...                       ...   
23444  7bxnKAamR3snQ1VGLuVfC1   
23445  5Aevni09Em4575077nkWHz   
23446  7ImMqPP3Q1yfUHvsdn7wEo   
23447  2m69mhnfQ1Oq6lGtXuYhgX   
23448  29zWqhca3zt5NsckZqDf6c   

                                              track_name      track_artist  \
0      I Don't Care (with Justin Bieber) - Loud Luxur...        Ed Sheeran   
1                        Memories - Dillon Francis Remix          Maroon 5   
2                        All the Time - Don Diablo Remix      Zara Larsson   
3                      Call You Mine - Keanu Silva Remix  The Chainsmokers   
4                Someone You Loved - Future Humans Remix     Lewis Capaldi   
...                                                  ...               ...   
23444               City Of Lights - Official Radio Edit 